In [1]:
from pyspark.sql import SparkSession

In [2]:
# Create a spark session (which will run spark jobs)
spark_gpu = (
    SparkSession.builder.appName("MAST30034_XGBOOST_GPU")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)


22/08/20 00:36:31 WARN Utils: Your hostname, DESKTOP-85B961I resolves to a loopback address: 127.0.1.1; using 192.168.178.215 instead (on interface eth0)
22/08/20 00:36:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/20 00:36:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import DoubleType, StringType, StructField, StructType

In [4]:
datapath = '../iris_temp.csv'

In [5]:
labelName = "class"
schema = StructType([StructField("sepal length", DoubleType(), True),
                      StructField("sepal width", DoubleType(), True),
                      StructField("petal length", DoubleType(), True),
                      StructField("petal width", DoubleType(), True),
                      StructField(labelName, StringType(), True)])

xgbInput = spark_gpu.read.option("header", "false").schema(schema).csv(datapath)

In [6]:
from pyspark.sql import Window
from pyspark.sql import functions
from pyspark.sql.functions import dense_rank

In [7]:
spec = Window.orderBy(labelName)

train, test = xgbInput.withColumn("tmpClassName", dense_rank().over(spec) - 1).drop(labelName).withColumnRenamed("tmpClassName", labelName).randomSplit(weights=[0.7,0.3], seed=1)
train.show(5)

22/08/20 00:36:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:36:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:36:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/20 00:36:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:36:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------------+-----------+------------+-----------+-----+
|sepal length|sepal width|petal length|petal width|class|
+------------+-----------+------------+-----------+-----+
|         4.3|        3.0|         1.1|        0.1|    0|
|         4.4|        2.9|         1.4|        0.2|    0|
|         4.4|        3.0|         1.3|        0.2|    0|
|         4.4|        3.2|         1.3|        0.2|    0|
|         4.6|        3.2|         1.4|        0.2|    0|
+------------+-----------+------------+-----------+-----+
only showing top 5 rows



In [8]:
from xgboost import XGBClassifier

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [9]:
xgbParam = {"objective" : "multi:softprob",
            "num_class" : 3,
            "num_round" : 100,
            "tree_method" : "gpu_hist",
            "num_workers" : 1}

featureNames = schema.names[:-1]

In [10]:
featureNames

['sepal length', 'sepal width', 'petal length', 'petal width']

XGBoostError: sklearn needs to be installed in order to use this module

In [1]:
import sklearn

In [11]:
xgbClassifier = XGBClassifier(xgbParam).setFeaturesCol(featureNames).setLabelCol(labelName)

/usr/local/lib/python3.8/dist-packages/xgboost/core.py:525: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


AttributeError: 'XGBClassifier' object has no attribute 'setFeaturesCol'

In [12]:
model = XGBClassifier(objective=multi:softprob,
        num_class=3,
        num_round=100,
        tree_method="gpu_hist",
        num_workers=1)

SyntaxError: invalid syntax (642779076.py, line 1)

In [13]:
xgbClassifier = XGBClassifier(xgbParam)

/usr/local/lib/python3.8/dist-packages/xgboost/core.py:525: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


In [16]:
train.show()
#xtrain = train.select(['sepal length', 'sepal width', 'petal length', 'petal width'])
#ytrain = train.select('class')

22/08/20 00:44:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------------+-----------+------------+-----------+-----+
|sepal length|sepal width|petal length|petal width|class|
+------------+-----------+------------+-----------+-----+
|

In [17]:
xgbClassifier.fit(xtrain, ytrain, eval_metric="rmse", verbose=True)

22/08/20 00:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [+-----+
 |class|
 +-----+
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    1|
 |    1|
 |    0|
 |    0|
 |    0|
 +-----+
 only showing top 20 rows]

In [19]:
params = {'max_depth': 50, 'n_estimators':80, 'learning_rate':0.1, 'colsample_bytree':7, 'gamma':0, 'reg_alpha':4, 'objective':'binary:logistic', 'eta':0.3, 'silent':1, 'subsample':0.8, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor',}

In [21]:
xgb_model = XGBClassifier(**params).fit(xtrain, ytrain)

22/08/20 00:50:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:50:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:50:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:50:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/20 00:50:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [+-----+
 |class|
 +-----+
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    0|
 |    1|
 |    1|
 |    0|
 |    0|
 |    0|
 +-----+
 only showing top 20 rows]

In [23]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer

featuresCols = xgbInput.columns
featuresCols = featuresCols.remove('class')

In [24]:
vectorAssembler = VectorAssembler(inputCols = featuresCols, outputCol='rawFeatures')


In [ ]:
vectorAssembler.fit()